In [37]:
#Writing a Neural Network Class Using PyTorch

import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
        
class NeuralNetwork:
    def __init__(self, neuronsperlayer, learning_rate): #Sets Up the Neural Network
        super().__init__()
        self.device = torch.device("cpu")
        self.noflayers = len(neuronsperlayer)
        layers = []
        
        for i in range(self.noflayers-1):
            layers.append(nn.Linear(neuronsperlayer[i], neuronsperlayer[i+1]))
            layers.append(nn.Sigmoid())
        
        self.net = nn.Sequential(*layers).to(self.device)
        
        def init_weights(m):
            if isinstance(m, nn.Linear):
                init.xavier_uniform_(m.weight, gain=init.calculate_gain("sigmoid"))
                init.zeros_(m.bias)

        self.net.apply(init_weights)
        
        self.loss_function = nn.MSELoss(reduction="sum")
        self.optimizer = optim.SGD(self.net.parameters(), lr=learning_rate, momentum=0)
        
    def forward(self, input_layer): #Forward Pass through Neural Network
        input_layer = torch.as_tensor(input_layer, dtype=torch.float32, device=self.device)
        return self.net(input_layer)
    
    def cost(self, output_layer, target): #Cost Function
        return self.loss_function(output_layer, target)
    
    def backward(self, preds, targets):
        loss = self.loss_function(preds, targets)
        loss.backward()
        return loss.item()
    
    def update(self): #Updating the Weights and Biases of Neural Network
        self.optimizer.step()         
        self.optimizer.zero_grad() 
            
    def learn(self, training_data, validation_data, epochs): #Stochastic Gradient Descent 
        training_data = torch.as_tensor(training_data, dtype=torch.float32, device=self.device)
        validation_data = torch.as_tensor(validation_data, dtype=torch.float32, device=self.device)
        
        if validation_data.dim() == 1:
            validation_data = validation_data.unsqueeze(1)
        
        data_size = training_data.size(0)
        
        for epoch in range(1, epochs + 1):
            permutation = torch.randperm(data_size, device=self.device)
            total_loss = 0.0
            data = list(zip(training_data, validation_data))
            np.random.shuffle(data)
            
            for index in permutation:
                training_batch = training_data[index].unsqueeze(0)
                validation_batch = validation_data[index].unsqueeze(0)

                out = self.forward(training_batch)
                total_loss += self.backward(out, validation_batch)
                self.update()

            if (epoch % 100 == 0 or epoch == 1):
                average_loss = total_loss / data_size
                print(f"Epoch {epoch:4d}/{epochs} — Average Loss: {average_loss:.4f}")


In [38]:
#Testing Neural Network Class by Training it to be an AND Gate

#Data
training_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
validation_data = np.array([0, 0, 0, 1])

#Training Neural Network
epochs = 500
learning_rate = 0.5
NN = NeuralNetwork([2, 2, 1], learning_rate)
NN.learn(training_data, validation_data, epochs)
print()

#Checking that the Neural Network works as an AND Gate
for x, y in zip(training_data, validation_data):
    output_layer = NN.forward(x)                       
    output_value = output_layer.detach().cpu().numpy().ravel()[0]
    prediction   = int(np.round(output_value))    
    input_layer = x.ravel().astype(int).tolist()
    target = int(y.ravel()[0])
    
    print(f"Input: {input_layer}, Ouput: {output_value:.3f} --> Rounds to: {prediction}, Expected: {target}")

Epoch    1/500 — Average Loss: 0.4003
Epoch  100/500 — Average Loss: 0.0912
Epoch  200/500 — Average Loss: 0.0124
Epoch  300/500 — Average Loss: 0.0048
Epoch  400/500 — Average Loss: 0.0028
Epoch  500/500 — Average Loss: 0.0019

Input: [0, 0], Ouput: 0.003 --> Rounds to: 0, Expected: 0
Input: [0, 1], Ouput: 0.044 --> Rounds to: 0, Expected: 0
Input: [1, 0], Ouput: 0.043 --> Rounds to: 0, Expected: 0
Input: [1, 1], Ouput: 0.939 --> Rounds to: 1, Expected: 1
